In [1]:
import plotly.express as px
import pandas as pd
import sqlite3

In [2]:
# Create a connection to the database
conn = sqlite3.connect("C:/Users/alexp/Documents/GTM/db.sqlite3")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute SQL queries to extract tables
cursor.execute("SELECT * FROM workout_workout")
workout_data = cursor.fetchall()

cursor.execute("SELECT * FROM workout_exercise")
exercise_data = cursor.fetchall()

# Convert the data to a pandas DataFrame
workout_df = pd.DataFrame(workout_data, columns=["id", "sets", "reps", "weight", "exercise", "date"])
exercise_df = pd.DataFrame(exercise_data, columns=["id", "name", "category"])

# Close the cursor and connection
cursor.close()
conn.close()

In [3]:
workout_df.drop(columns=["id"], inplace=True)

In [4]:
merged_df = workout_df.merge(exercise_df, left_on='exercise', right_on='id')

In [5]:
merged_df.sort_values(by='date', inplace=True)
merged_df.drop(columns=["exercise", "id", "category"], inplace=True)
merged_df.rename(columns={'name': 'exercise'}, inplace=True)

In [6]:
fig = px.bar(merged_df['exercise'].value_counts(), x=merged_df['exercise'].value_counts().index, y=merged_df['exercise'].value_counts().values, template="plotly_dark")
fig.update_layout(title="# of workouts over the last year", xaxis_title="Exercise Name", yaxis_title="# of times completed", xaxis_tickangle=-45)
fig.add_hline(y=merged_df['exercise'].value_counts().mean(), line_dash="dot", line_color="red", annotation_text="Average", annotation_position="bottom right")
fig.show()